# Part 3: RF Model Development

In [121]:
import pandas as pd
ml_df = pd.read_csv('data/racedata_2017-2022_ETL.csv', index_col = 0)

In [122]:
ml_df = ml_df.dropna()

In [123]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

working_directory = 'D:/machine_learning/DFS/nascar/'
data_dir = 'data/'

# needs hyperparameters

dcols = [
    'Pos',
     'St',
     '#',
     'Driver',
     'Laps',
     'Led',
     'Pts',
     'PPts',
     'Track',
     'race_ID',
     'FP',
     'place_diff',
     'Date',
     'Date_dtobj',
     'Year',
     'DNF',
     'Win',
     'T5',
     'T10',
     'T20',
     'track_type'
]

X = ml_df.drop(dcols, axis = 1)
Y = ml_df['FP']

from sklearn.model_selection import train_test_split

#Create Training and Testing DataSets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42)

X_train.reset_index(inplace = True, drop=True)
X_test.reset_index(inplace = True, drop=True)
Y_train.reset_index(inplace = True, drop=True)
Y_test.reset_index(inplace = True, drop=True)

print('Training set size:', len(X_train))
print('Testing set size:', len(X_test))

pred_df = pd.concat([X_test, Y_test], axis = 1)

print('\nNum Possible Features:',len(X_train.columns.tolist()))

Training set size: 6234
Testing set size: 2078

Num Possible Features: 67


In [124]:
# get statuses for comparison
train_status = X_train['Status']
test_status = X_test['Status']

X_train.drop(['Status'], axis = 1, inplace = True)
X_test.drop(['Status'], axis = 1, inplace = True)

pred_df['Status'] = test_status

C:\Users\Greg\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [125]:
# standardize data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.compose import ColumnTransformer

# get numerical features from the features dataframe and store numerical columns in a variable
numerical_features = X_train.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns
 
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

# fit columntransformer to the training data and transform it
X_train_scaled = ct.fit_transform(X_train)

# fit columntransformer to the testing data and transform it
X_test_scaled = ct.transform(X_test)

# convert to pd dataframes
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_test_scaled = pd.DataFrame(X_train_scaled, columns = X_test.columns)

In [127]:
X_train_scaled

,LL_yr_cumsum,LC_yr_cumsum,DNF_yr_cumsum,Win_yr_cumsum,T5_yr_cumsum,T10_yr_cumsum,T20_yr_cumsum,FinPos_mean_L3,StrtPos_mean_L3,PlaceDiff_mean_L3,LL_sum_L3,LL_mean_L3,LC_sum_L3,LC_mean_L3,Pts_sum_L3,Pts_mean_L3,PPts_sum_L3,PPts_mean_L3,FP_sum_L3,FP_mean_L3,DNF_sum_L3,DNF_sum_L10,Win_sum_L3,T5_sum_L3,T10_sum_L3,T20_sum_L3,Win_sum_L10,T5_sum_L10,T10_sum_L10,T20_sum_L10,FinPos_mean_L3_track,StrtPos_mean_L3_track,PlaceDiff_mean_L3_track,LL_sum_L3_track,LL_mean_L3_track,LC_sum_L3_track,LC_mean_L3_track,Pts_sum_L3_track,Pts_mean_L3_track,PPts_sum_L3_track,PPts_mean_L3_track,FP_sum_L3_track,FP_mean_L3_track,DNF_sum_L3_track,Win_sum_L3_track,T5_sum_L3_track,T10_sum_L3_track,T20_sum_L3_track,FinPos_mean_L3_tt,StrtPos_mean_L3_tt,PlaceDiff_mean_L3_tt,LL_sum_L3_tt,LL_mean_L3_tt,LC_sum_L3_tt,LC_mean_L3_tt,Pts_sum_L3_tt,Pts_mean_L3_tt,PPts_sum_L3_tt,PPts_mean_L3_tt,FP_sum_L3_tt,FP_mean_L3_tt,DNF_sum_L3_tt,Win_sum_L3_tt,T5_sum_L3_tt,T10_sum_L3_tt,T20_sum_L3_tt
0,-0.488737,-0.305571,-0.687274,-0.421026,-0.655717,-0.485303,-0.406760,0.443735,0.094085,-0.448399,-0.394155,-0.394093,-0.756103,-0.805360,-0.423888,-0.432568,-0.314496,-0.314009,-0.751514,-0.801173,0.912111,-0.293056,-0.269742,-0.560113,0.250758,-0.478027,-0.393498,-0.715303,-0.599061,0.250758,-0.358560,0.082279,0.696784,-0.309283,-0.304269,0.325581,0.707837,0.470334,0.870019,-0.250452,-0.243807,0.511497,0.996399,-0.471388,-0.216976,-0.443084,0.552535,0.912021,0.772489,0.179367,-0.787212,-0.374849,-0.371507,-0.036854,-0.129560,-0.535443,-0.577473,-0.293484,-0.292616,-0.422855,-0.531861,1.003053,-0.252433,-0.539774,-0.774935,-0.387907
1,-0.477410,0.270799,0.310506,-0.421026,-0.655717,-0.306698,0.201302,0.747097,-0.307039,-1.457496,-0.394155,-0.394093,0.084161,0.055167,-0.610100,-0.619051,-0.314496,-0.314009,-0.677959,-0.725619,0.912111,-0.293056,-0.269742,-0.560113,-0.821412,-0.478027,-0.393498,-0.715303,-0.599061,-0.821412,0.717991,1.045534,0.561371,-0.309283,-0.304269,0.271074,0.631905,-0.245246,-0.098649,-0.250452,-0.243807,0.133656,0.462926,-0.471388,-0.216976,-0.443084,-0.620661,-0.909684,-0.159264,0.211132,0.529639,-0.374849,-0.371507,-0.763404,-0.861380,0.152795,0.109553,-0.293484,-0.292616,-0.171780,-0.272870,-0.631299,-0.252433,-0.539774,0.279857,0.487306
2,-0.462308,0.636707,0.310506,-0.421026,-0.655717,-0.663907,0.201302,-0.087148,0.428356,0.762516,-0.394155,-0.394093,-0.849900,-0.901419,-0.051463,-0.059603,-0.314496,-0.314009,-0.255543,-0.291722,-0.690059,-0.293056,-0.269742,-0.560113,-0.821412,1.334086,-0.393498,-0.715303,-0.997541,-0.821412,0.545743,0.878012,0.561371,-0.289202,-0.287460,1.455994,1.073193,0.265883,0.039732,-0.250452,-0.243807,1.149687,0.791555,-0.471388,-0.216976,-0.443084,-0.620661,0.001169,0.116811,0.846448,1.094003,-0.374849,-0.371507,-0.208635,-0.302587,-0.051128,-0.094010,-0.293484,-0.292616,0.120590,0.028719,-0.631299,-0.252433,-0.539774,-0.774935,0.487306
3,-0.492512,-1.500034,0.310506,-0.421026,-0.655717,-0.842511,-1.136435,1.846785,1.531449,-0.145671,-0.394155,-0.394093,-2.405366,-2.494396,-1.514562,-1.524823,-0.314496,-0.314009,-2.352912,-2.446097,2.514282,1.407979,-0.269742,-0.560113,-0.821412,-1.384083,-0.393498,-0.715303,-0.997541,-0.821412,-1.262863,-1.216022,0.019716,-0.309283,-0.304269,-1.148459,-1.345642,-0.960826,-1.067316,-0.250452,-0.243807,-1.200135,-1.420250,-0.471388,-0.216976,-0.443084,-0.620661,-0.909684,1.773261,1.545296,-0.128787,-0.374849,-0.371507,-1.599782,-1.703823,-1.376623,-1.417172,-0.293484,-0.292616,-1.703005,-1.852378,2.637405,-0.252433,-0.539774,-0.774935,-1.263121
4,-0.484961,-0.644894,-1.186164,-0.421026,-0.379481,-0.485303,-0.649985,-0.580112,0.394929,1.367974,-0.394155,-0.394093,1.127651,1.123823,0.294361,0.286721,-0.314496,-0.314009,1.207151,1.210727,-0.690059,-1.143573,-0.269742,-0.560113,0.250758,1.334086,-0.393498,-0.149128,-0.200582,0.250758,-1.262863,-1.216022,0.019716,-0.309283,-0.304269,-1.148459,-1.345642,-0.960826,-1.067316,-0.250452,-0.243807,-1.200135,-1.420250,-0.471388,-0.216976,-0.443084,-0.620661,-0

In [128]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

working_directory = 'D:/machine_learning/DFS/nascar/ML'
os.chdir(working_directory)

# needs hyperparameters
def lgbm_mod(): return LGBMRegressor(random_state = 1, n_estimators = 1000, learning_rate = 0.01, n_jobs = -1)
def xgb_mod(): return xgb.XGBRegressor(random_state = 1)
def rf_mod(): return RandomForestRegressor(random_state = 1, n_jobs = -1, n_estimators = 200)
def sgd_mod(): return SGDRegressor(max_iter=100, tol=1e-3, random_state = 1) # won't work with feature importance
def svm_mod(): return SVR(C=1.0, epsilon=0.2) # won't work with feature importance
def neigh_mod(): return KNeighborsRegressor()
def dt_mod(): return DecisionTreeRegressor(random_state=0)

""" MODEL SELECTION """

model = rf_mod()

"""                 """

#print possible features
print('possible features:', X_train_scaled.columns.tolist(), '\n')

# Fit model, make predictions with all features
model.fit(X_train_scaled, Y_train)

preds_all = model.predict(X_test)

pdf = pred_df[['Status', 'FP']].copy()
pdf['Pred_FP_all'] = preds_all

# save the initial model to disk
filename = 'models/nascar_model_allfeats.pkl'
joblib.dump(model, filename)

# get top 50 features
dset = pd.DataFrame({'attr':X_train.columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr50 = dset['attr'][0:50].tolist()

# Using Top 50 Features, Find Top 30 Features
model.fit(X_train[attr50], Y_train)
dset = pd.DataFrame({'attr':X_train[attr50].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr30 = dset['attr'][0:30].tolist()

# Using Top 30 Features, Find Top 20 Features
model.fit(X_train[attr30], Y_train)
dset = pd.DataFrame({'attr':X_train[attr30].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr20 = dset['attr'][0:20].tolist()

#Perform RFE (recursive feature elimination) using Top 20 Features, To Find Top 15
rfe_model = RFE(model, n_features_to_select = 15)
rfe_model.fit(X_train[attr20], Y_train)
dset = pd.DataFrame({'attr':X_train[attr20].columns.tolist(),'importance':rfe_model.ranking_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
cols15 = dset[dset['importance']==1]['attr'].tolist()

#Perform RFE using Top 15 Features, To Find Top 10
rfe_model = RFE(model, n_features_to_select = 10)
rfe_model.fit(X_train[cols15], Y_train)
dset = pd.DataFrame({'attr':X_train[cols15].columns.tolist(),'importance':rfe_model.ranking_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
cols10 = dset[dset['importance']==1]['attr'].tolist()

print('T50 features', attr50, '\n')
print('T30 features', attr30, '\n')
print('T20 features', attr20, '\n')
print('T15 features',cols15, '\n')
print('T10 features',cols10, '\n')

model.fit(X_train[attr50], Y_train)
preds50 = model.predict(X_test[attr50])
filename = 'models/nascar_model_50feats.pkl'
joblib.dump(model, filename)

model.fit(X_train[attr30], Y_train)
preds30 = model.predict(X_test[attr30])
filename = 'models/nascar_model_30feats.pkl'
joblib.dump(model, filename)

model.fit(X_train[attr20], Y_train)
preds20 = model.predict(X_test[attr20])
filename = 'models/nascar_model_20feats.pkl'
joblib.dump(model, filename)

model.fit(X_train[cols15], Y_train)
preds15 = model.predict(X_test[cols15])
filename = 'models/nascar_model_15feats.pkl'
joblib.dump(model, filename)

model.fit(X_train[cols10], Y_train)
preds10 = model.predict(X_test[cols10])
filename = 'models/nascar_model_10feats.pkl'
joblib.dump(model, filename)


pdf['Pred_FP_50'] = preds50
pdf['Pred_FP_30'] = preds30
pdf['Pred_FP_20'] = preds20
pdf['Pred_FP_15'] = preds15
pdf['Pred_FP_10'] = preds10

pdf.to_csv('predictions_50_30_20_15_10.csv')

feature_sets = ['all', '50', '30', '20', '15', '10']

mae_values = [
    "{:.2f}".format(mean_absolute_error(Y_test, preds_all)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds50)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds30)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds20)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds15)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds10))
]

results_df = pd.DataFrame({'Features' : feature_sets, 'MAE' : mae_values})

results_df.style.hide_index()
results_df.to_clipboard()
display(results_df)

possible features: ['LL_yr_cumsum', 'LC_yr_cumsum', 'DNF_yr_cumsum', 'Win_yr_cumsum', 'T5_yr_cumsum', 'T10_yr_cumsum', 'T20_yr_cumsum', 'FinPos_mean_L3', 'StrtPos_mean_L3', 'PlaceDiff_mean_L3', 'LL_sum_L3', 'LL_mean_L3', 'LC_sum_L3', 'LC_mean_L3', 'Pts_sum_L3', 'Pts_mean_L3', 'PPts_sum_L3', 'PPts_mean_L3', 'FP_sum_L3', 'FP_mean_L3', 'DNF_sum_L3', 'DNF_sum_L10', 'Win_sum_L3', 'T5_sum_L3', 'T10_sum_L3', 'T20_sum_L3', 'Win_sum_L10', 'T5_sum_L10', 'T10_sum_L10', 'T20_sum_L10', 'FinPos_mean_L3_track', 'StrtPos_mean_L3_track', 'PlaceDiff_mean_L3_track', 'LL_sum_L3_track', 'LL_mean_L3_track', 'LC_sum_L3_track', 'LC_mean_L3_track', 'Pts_sum_L3_track', 'Pts_mean_L3_track', 'PPts_sum_L3_track', 'PPts_mean_L3_track', 'FP_sum_L3_track', 'FP_mean_L3_track', 'DNF_sum_L3_track', 'Win_sum_L3_track', 'T5_sum_L3_track', 'T10_sum_L3_track', 'T20_sum_L3_track', 'FinPos_mean_L3_tt', 'StrtPos_mean_L3_tt', 'PlaceDiff_mean_L3_tt', 'LL_sum_L3_tt', 'LL_mean_L3_tt', 'LC_sum_L3_tt', 'LC_mean_L3_tt', 'Pts_sum_L3_t

,Features,MAE
0,all,20.76
1,50,12.89
2,30,12.98
3,20,13.02
4,15,12.90
5,10,13.13


In [112]:
pdf

,Status,FP,Pred_FP_all,Pred_FP_50,Pred_FP_30,Pred_FP_20,Pred_FP_15,Pred_FP_10
0,running,44.5,41.2170,35.1475,34.5690,34.9915,35.1180,36.0010
1,running,74.4,73.7685,63.9265,65.1180,66.6040,67.0100,59.9400
2,crash,20.0,62.4925,51.6150,49.9260,54.5045,52.8350,54.2920
3,running,77.0,40.2835,59.9685,58.9900,57.5540,58.0545,60.9680
4,running,72.7,72.4115,55.5400,54.2515,54.3110,54.0315,52.4095
...,...,...,...,...,...,...,...,...
2073,running,82.5,54.2185,43.9155,43.6950,45.1135,46.4835,47.8185
2074,running,37.2,83.3100,62.1710,61.6550,67.1800,68.5485,67.6495
2075,running,61.8,39.7285,44.9425,44.3685,44.0030,44.1675,42.6375
2076,engine,4.9,82.3420,61.1035,59.8680,55.3640,51.3490,58.3060


In [381]:
pdf.loc[pdf['status'] == 'running'].describe()

,actual,predicted,abs. error
count,1797.000000,1797.000000,1797.000000
mean,53.519421,49.623383,11.473874
std,18.939117,14.068583,9.130235
min,0.000000,12.833677,0.002911
25%,39.700000,39.377014,4.421690
50%,53.400000,48.874165,9.301463
75%,66.000000,58.573917,16.168793
max,138.600000,108.717484,62.869211


In [382]:
pdf.loc[pdf['status'] != 'running'].describe()

,actual,predicted,abs. error
count,281.00000,281.000000,281.000000
mean,18.48363,41.552753,24.411830
std,14.62147,12.253642,15.295260
min,0.00000,13.525291,0.058690
25%,6.10000,32.670452,12.266153
50%,15.90000,39.381245,23.028477
75%,28.10000,48.775669,34.328568
max,73.60000,79.006569,70.062119


In [383]:
pdf.head(n=20)

,actual,predicted,abs. error,status
0,44.5,32.008884,12.491116,running
1,74.4,82.746796,8.346796,running
2,20.0,54.328568,34.328568,crash
3,77.0,57.456577,19.543423,running
4,72.7,52.579689,20.120311,running
5,56.8,56.631878,0.168122,running
6,37.2,38.757439,1.557439,running
7,44.9,35.598537,9.301463,running
8,40.0,39.028259,0.971741,running
9,39.7,34.268368,5.431632,electrical


In [384]:
pdf.describe()

,actual,predicted,abs. error
count,2078.000000,2078.000000,2078.000000
mean,48.781665,48.532024,13.223424
std,21.968089,14.107090,11.099790
min,0.000000,12.833677,0.002911
25%,33.900000,38.127890,4.790457
50%,49.800000,47.544008,10.416541
75%,63.575000,57.595137,18.772615
max,138.600000,108.717484,70.062119


---
**need to develop predictors based on track type**

* assign track type for each race, since it's known
* for each driver and track type, calculate means/sums for L3 of that track type